# Flux.pl

The `Flux.pl` Perl script takes four input parameters:

`Flux.pl [input file] [output file] [bin width (s)] [geometry base directory]`

or, as invoked from the command line,

`$ perl ./perl/Flux.pl [input file] [output file] [bin width (s)] [geometry directory]`

## Input Parameters

* `[input file]`

`Flux.pl` expects the first non-comment line of the input file to begin with a string of the form `<DAQ ID>.<channel>`.  This is satisfied by threshold and wire delay files, as well as the outputs of data transformation scripts like `Sort.pl` and `Combine.pl` if their inputs are of the appropriate form.

If the input file doesn't meet this condition, `Flux.pl` -- specifically, the `all_geo_info{}` subroutine of `CommonSubs.pl` -- won't be able to load the appropriate geometry files and execution will fail (see the `[geometry directory]` parameter below).

* `[output file]`

This is what the output file will be named.

* `[binWidth]`

In physical terms, cosmic ray _flux_ is the number of incident rays per unit area per unit time.  The `[binWidth]` parameter determines the "per unit time" portion of this quantity.  `Flux.pl` will sort the events in its input data into bins of the given time interval, returning the number of events per unit area recorded within each bin.

* `[geometry directory]`

With `[binWidth]` handling the "per unit time" portion of the flux calculation, the geometry file associated with each detector handles the "per unit area".  

`Flux.pl` expects geometry files to be stored in a directory structure of the form

```
geo/
├── 6119/
│   └── 6119.geo
├── 6148/
│   └── 6148.geo
└── 6203/
    └── 6203.geo
```

where each DAQ has its own subdirectory whose name is the DAQ ID, and each such subdirectory has a geometry file  whose name is given by the DAQ ID with the `.geo` extension.  The command-line argument in this case is `geo/`, the parent directory.  With this as the base directory, `Flux.pl` determines what geometry file to load by looking for the DAQ ID in the first line of data.  This is why, as noted above, the first non-comment line of `[input file]` must begin with `<DAQ ID>.<channel>`.

## Flux Input Files

As we mentioned above, threshold files have the appropriate first-line structure to allow `Flux.pl` to access geometry data for them.  So what does `Flux.pl` do when acting on a threshold file?

We'll test it using the threshold files `files/6148.2016.0109.0.thresh` and `files/6119.2016.0104.1.thresh` as input.  First, take a look at the files themselves so we know what the input looks like:

In [4]:
!head -10 files/6148.2016.0109.0.thresh

#$md5
#md5_hex(0)
#ID.CHANNEL, Julian Day, RISING EDGE(sec), FALLING EDGE(sec), TIME OVER THRESHOLD (nanosec), RISING EDGE(INT), FALLING EDGE(INT)
6148.4	2457396	0.5006992493422453	0.5006992493424479	17.51	4326041514317000	4326041514318750
6148.3	2457396	0.5006992493422887	0.5006992493424768	16.25	4326041514317375	4326041514319000
6148.2	2457396	0.5007005963399161	0.5007005963400029	7.49	4326053152376876	4326053152377625
6148.3	2457396	0.5007005963401910	0.5007005963404514	22.49	4326053152379250	4326053152381500
6148.4	2457396	0.5007005963401765	0.5007005963404658	25.00	4326053152379125	4326053152381624
6148.1	2457396	0.5014987243978154	0.5014987243980903	23.75	4332948978797125	4332948978799500
6148.2	2457396	0.5014987243980759	0.5014987243982495	15.00	4332948978799376	4332948978800875


In [5]:
!wc -l files/6148.2016.0109.0.thresh

6703 files/6148.2016.0109.0.thresh


In [6]:
!head -10 files/6119.2016.0104.1.thresh

#$md5
#md5_hex(0)
#ID.CHANNEL, Julian Day, RISING EDGE(sec), FALLING EDGE(sec), TIME OVER THRESHOLD (nanosec), RISING EDGE(INT), FALLING EDGE(INT)
6119.1	2457392	0.3721863017828993	0.3721863017831598	22.50	3215689647404250	3215689647406500
6119.3	2457392	0.3721863017829138	0.3721863017831598	21.25	3215689647404375	3215689647406500
6119.2	2457392	0.3721885846820747	0.3721885846822772	17.50	3215709371653125	3215709371654875
6119.4	2457392	0.3721885846820747	0.3721885846822917	18.75	3215709371653125	3215709371655000
6119.4	2457392	0.3721901866161603	0.3721901866163773	18.75	3215723212363625	3215723212365500
6119.1	2457392	0.3721901866161748	0.3721901866164496	23.75	3215723212363750	3215723212366125
6119.1	2457392	0.3721903650327546	0.3721903650329427	16.25	3215724753883000	3215724753884625


In [7]:
!wc -l files/6119.2016.0104.1.thresh

181008 files/6119.2016.0104.1.thresh


(remember, `wc -l` returns a count of the number of lines in the file).  These look like fairly standard threshold files.  Now we'll see what `Flux.pl` does with them.

## Flux Output

Below is the output generated by `Flux.pl` using the threshold files `6148.2016.0109.0.thresh` and `6119.2016.0104.1.thresh` (separately) as input:

```
$ perl ./perl/Flux.pl files/6148.2016.0109.0.thresh outputs/ThreshFluxOut6148_01 600 geo/
$ head -15 outputs/ThreshFluxOut6148_01 
#cf12d07ed2dfe4e4c0d52eb663dd9956
#md5_hex(1536259294 1530469616 files/6148.2016.0109.0.thresh outputs/ThreshFluxOut6148_01 600 geo/)
01/09/2016 00:06:00 59.416172 8.760437
01/09/2016 00:16:00 63.291139 9.041591
01/09/2016 00:26:00 71.041075 9.579177
01/09/2016 00:36:00 50.374580 8.066389
01/09/2016 00:46:00 55.541204 8.469954
01/09/2016 00:56:00 73.624386 9.751788
01/09/2016 01:06:00 42.624645 7.419998
01/09/2016 01:16:00 54.249548 8.370887
01/09/2016 01:26:00 45.207957 7.641539
01/09/2016 01:36:00 42.624645 7.419998
01/09/2016 01:46:00 65.874451 9.224268
01/09/2016 01:56:00 59.416172 8.760437
01/09/2016 02:06:00 94.290881 11.035913
```

```
$ perl ./perl/Flux.pl files/6119.2016.0104.1.thresh outputs/ThreshFluxOut6119_01 600 geo/
$ head -15 outputs/ThreshFluxOut6119_01 
#84d0f02f26edb8f59da2d4011a27389d
#md5_hex(1536259294 1528996902 files/6119.2016.0104.1.thresh outputs/ThreshFluxOut6119_01 600 geo/)
01/04/2016 21:00:56 12496.770860 127.049313
01/04/2016 21:10:56 12580.728494 127.475379
01/04/2016 21:20:56 12929.475588 129.230157
01/04/2016 21:30:56 12620.769827 127.678079
01/04/2016 21:40:56 12893.309222 129.049289
01/04/2016 21:50:56 12859.726169 128.881113
01/04/2016 22:00:56 12782.226815 128.492174
01/04/2016 22:10:56 12520.020666 127.167443
01/04/2016 22:20:56 12779.643503 128.479189
01/04/2016 22:30:56 12746.060449 128.310265
01/04/2016 22:40:56 12609.144924 127.619264
01/04/2016 22:50:56 12372.771894 126.417419
01/04/2016 23:00:56 12698.269181 128.069490
```

`Flux.pl` seems to give reasonable output with a threshold file as input, provided the DAQ has a geometry file that's up to standards.  Can we interpret the output?  Despite the lack of a header line, some reasonable inferences will make it clear.

The first column is clearly the date that the data was taken, and in both cases it agrees with the date indicated by the threshold file's filename.

The second column is clearly time-of-day values, but what do they mean?  We might be tempted to think of them as the full-second portion of cosmic ray event times, but we note in both cases that they occur in a regular pattern of exactly every ten minutes.  Of course, that happens to be exactly what we selected as the `binWidth` parameter, 600s = 10min.  These are the time bins into which the cosmic ray event data is organized.

Since we're calculating flux -- muon strikes per unit area per unit time -- we expect the flux count itself to be included in the data, and in fact this is what the third column is, in units of $events/m^2/min$.  Note that the "$/min$" part is *always* a part of the units of the third column, no matter what the size of the time bins we selected.

Finally, when doing science, having a measurement means having uncertainty.  The fourth column is the obligatory statistical uncertainty in the flux.

## An exercise in statistical uncertainty

The general formula for flux $\Phi$ is

$$\Phi = \frac{N}{AT}$$

where $N$ is the number of incident events, $A$ is the cross-sectional area over which the flux is measured, and $T$ is the time interval over which the flux is measured.

By the rule of quadrature for propagating uncertainties,

$$\frac{\delta \Phi}{\Phi} \approx \frac{\delta N}{N} + \frac{\delta A}{A} + \frac{\delta T}{T}$$

Here, $N$ is the raw count of muon hits in the detector, an integer with a standard statistical uncertainty of $\sqrt{N}$.

In our present analysis, errors in the bin width and detector area are negligible compared to the statistical fluctuation of cosmic ray muons.  Thus, we'll take $\delta A \approx \delta T \approx 0$ to leave

$$\delta \Phi \approx \frac{\delta N}{N} \Phi = \frac{\Phi}{\sqrt{N}}$$

Rearranging this a bit, we find that we should be able to calculate the exact number of muon strikes for each time bin as

$$N \approx \left(\frac{\Phi}{\delta\Phi}\right)^2.$$

Let's see what happens when we apply this to the data output from `Flux.pl`.  For the 6148 data with `binWidth=600`, we find

```
Date        Time           Phi          dPhi        (Phi/dPhi)^2
01/09/16	12:06:00 AM	59.416172	8.760437	45.999996082
01/09/16	12:16:00 AM	63.291139	9.041591	49.0000030968
01/09/16	12:26:00 AM	71.041075	9.579177	54.9999953935
01/09/16	12:36:00 AM	50.37458	 8.066389	38.9999951081
01/09/16	12:46:00 AM	55.541204	8.469954	43.0000020769
01/09/16	12:56:00 AM	73.624386	9.751788	57.000001784
01/09/16	01:06:00 AM	42.624645	7.419998	33.0000025577
01/09/16	01:16:00 AM	54.249548	8.370887	41.999999903
01/09/16	01:26:00 AM	45.207957	7.641539	35.0000040418
01/09/16	01:36:00 AM	42.624645	7.419998	33.0000025577
01/09/16	01:46:00 AM	65.874451	9.224268	51.00000197
01/09/16	01:56:00 AM	59.416172	8.760437	45.999996082
01/09/16	02:06:00 AM	94.290881	11.035913   72.9999984439
```

The numbers we come up with are in fact integers to an excellent approximation!

---

### Exercise 1

**A)** Using the data table above, round the `(Phi/dPhi)^2` column to the nearest integer, calling it `N`.  With $\delta N = \sqrt{N}$, calculate $\frac{\delta N}{N}$ for each row in the data.

**B)** Using your knowledge of the cosmic ray muon detector, estimate the uncertainty $\delta A$ in the detector area $A$ and the uncertainty $\delta T$ in the time bin $T$ given as the input `binWidth` parameter.  Calculate $\frac{\delta A}{A}$ and $\frac{\delta T}{T}$ for this analysis.

**C)** Considering the results of **A)** and **B)**, do you think our previous assumption that $\frac{\delta A}{A} \approx 0$ and $\frac{\delta T}{T} \approx 0$ compared to $\frac{\delta N}{N}$ is justified?

---

### Additional Exercises

* Do the number of counts $N$ in one `binWidth=600s` bin match the sum of counts in the ten corresponding `binWidth=60s` bins?

* Considering raw counts, do you think the "zero" bins in the above analyses are natural fluctuations in cosmic ray muon strikes?

* Do the flux values shown above reasonably agree with the known average CR muon flux at sea level?  If "no," what effects do you think might account for the difference?

---

We can dig more information out of the `Flux.pl` output by returning to the definition of flux

$$\Phi = \frac{N}{AT}.$$

Now that we know $N$ for each data point, and given that we know the bin width $T$ because we set it for the entire analysis, we should be able to calculate the area of the detector as

$$A = \frac{N}{\Phi T}$$

One important comment: `Flux.pl` gives flux values in units of `events/m^2/min` - note the use of minutes instead of seconds.  When substituting a numerical value for $T$, we must convert the command line parameter `binWidth=600` from $600s$ to $10min$.

When we perform this calculation, we find consistent values for $A$:

```
Date        Time           Phi          dPhi        N=(Phi/dPhi)^2   A=N/Phi T
01/09/16	12:06:00 AM	59.416172	8.760437	45.999996082	 0.0774199928
01/09/16	12:16:00 AM	63.291139	9.041591	49.0000030968	0.0774200052
01/09/16	12:26:00 AM	71.041075	9.579177	54.9999953935	0.0774199931
01/09/16	12:36:00 AM	50.37458	 8.066389	38.9999951081	0.0774199906
01/09/16	12:46:00 AM	55.541204	8.469954	43.0000020769	0.0774200035
01/09/16	12:56:00 AM	73.624386	9.751788	57.000001784	 0.0774200029
01/09/16	01:06:00 AM	42.624645	7.419998	33.0000025577	0.0774200056
01/09/16	01:16:00 AM	54.249548	8.370887	41.999999903	 0.0774199997
01/09/16	01:26:00 AM	45.207957	7.641539	35.0000040418	0.0774200083
01/09/16	01:36:00 AM	42.624645	7.419998	33.0000025577	0.0774200056
01/09/16	01:46:00 AM	65.874451	9.224268	51.00000197	  0.077420003
01/09/16	01:56:00 AM	59.416172	8.760437	45.999996082	 0.0774199928
01/09/16	02:06:00 AM	94.290881	11.035913   72.9999984439	0.0774199983
```

In fact, the area of one standard 6000-series QuarkNet CRMD detector panel is $0.07742m^2$.

It's important to note that we're reversing only the calculations, not the physics!  That is, we find $A=0.07742m^2$ because that's the value stored in the `6248.geo` file, not because we're able to determine the actual area of the detector panel from the `Flux.pl` output data using physical principles.

## Testing binWidth

To verify that the third-column flux values behave as expected, we can run a quick check by manipulating the `binWidth` parameter.  We'll run `Flux.pl` on the above two threshold files again, but this time we'll reduce `binWidth` by a factor of 10:

```
$ perl ./perl/Flux.pl files/6148.2016.0109.0.thresh outputs/ThreshFluxOut6148_02 60 geo/
```

In [8]:
!head -15 outputs/ThreshFluxOut6148_02 

#d28fbf9f1f5e4939813797ac0d28f3db
#md5_hex(1536259294 1530469616 files/6148.2016.0109.0.thresh outputs/ThreshFluxOut6148_02 60 geo/)
01/09/2016 00:01:30 64.582795 28.882304
01/09/2016 00:02:30 77.499354 31.638979
01/09/2016 00:13:30 25.833118 18.266773
01/09/2016 00:14:30 25.833118 18.266773
01/09/2016 00:15:30 142.082149 42.839380
01/09/2016 00:16:30 116.249031 38.749677
01/09/2016 00:17:30 77.499354 31.638979
01/09/2016 00:18:30 90.415913 34.174003
01/09/2016 00:19:30 51.666236 25.833118
01/09/2016 00:23:30 103.332472 36.533546
01/09/2016 00:24:30 64.582795 28.882304
01/09/2016 00:25:30 103.332472 36.533546
01/09/2016 00:26:30 90.415913 34.174003


```
$ perl ./perl/Flux.pl files/6119.2016.0104.1.thresh outputs/ThreshFluxOut6119_02 60 geo/
```

In [9]:
!head -15 outputs/ThreshFluxOut6119_02 

#d20cb8a6a91adb6dd45998a811d75401
#md5_hex(1536259294 1528996902 files/6119.2016.0104.1.thresh outputs/ThreshFluxOut6119_02 60 geo/)
01/04/2016 20:56:26 12580.728494 403.112543
01/04/2016 20:57:26 12399.896668 400.204944
01/04/2016 20:58:26 12942.392147 408.865714
01/04/2016 20:59:26 12735.727202 405.588181
01/04/2016 21:00:26 11883.234306 391.778633
01/04/2016 21:01:26 12231.981400 397.485987
01/04/2016 21:02:26 12076.982692 394.959567
01/04/2016 21:03:26 12593.645053 403.319426
01/04/2016 21:04:26 12903.642470 408.253181
01/04/2016 21:05:26 12619.478171 403.732875
01/04/2016 21:06:26 12671.144407 404.558506
01/04/2016 21:07:26 12929.475588 408.661638
01/04/2016 21:08:26 13097.390855 411.306725


In the case of the 6148 data, our new fine-grained binning reveals some sparsity in the first several minutes of the data, as all of the bins between the `2:30` bin and the `13:30` bin are empty of muon events (and therefore not reported).  What happened here?  It's difficult to say -- under normal statistical variations, it's possible that there were simply no recorded events during these bins.  It's also possible that the experimenter adjusted the level of physical shielding around the detector during these times, or had a cable unplugged while troubleshooting.